
# SpaceX Falcon 9 First Stage Landing Prediction


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)



![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


In [1]:
import requests
from bs4 import BeautifulSoup
import unicodedata
import pandas as pd

In [2]:
def date_time(table_cells):
    """
    Extracts date and time from an HTML table cell.
    Input: A table data cell element.
    Returns: List of [date, time].
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    Extracts booster version from an HTML table cell.
    Input: A table data cell element.
    Returns: Booster version string.
    """
    out = ''.join([bv for i, bv in enumerate(table_cells.strings) if i % 2 == 0][0:-1])
    return out

def landing_status(table_cells):
    """
    Extracts landing status from an HTML table cell.
    Input: A table data cell element.
    Returns: Landing status string.
    """
    out = [i for i in table_cells.strings][0]
    return out

def get_mass(table_cells):
    """
    Extracts payload mass from an HTML table cell.
    Input: A table data cell element.
    Returns: Mass string (e.g., '500 kg') or 0 if not found.
    """
    mass = unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        kg_index = mass.find("kg")
        if kg_index != -1:
            new_mass = mass[0:kg_index + 2]
        else:
            new_mass = mass  # Take the whole string if 'kg' is not found
    else:
        new_mass = 0
    return new_mass

def extract_column_from_header(row):
    """
    Extracts column name from a table header cell.
    Input: A table header element (<th>).
    Returns: Cleaned column name string or None.
    """
    if row.br:
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
    column_name = ' '.join(row.contents)
    if not(column_name.strip().isdigit()):
        column_name = column_name.strip()
        return column_name
    

In [3]:
# URL of the Wikipedia page snapshot from June 9, 2021
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

In [4]:
# Perform HTTP GET request
response = requests.get(static_url)

# Create BeautifulSoup object
soup = BeautifulSoup(response.content, 'html.parser')

# Verify by printing the page title
print(soup.title)

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


In [5]:
# Find all tables with the specified class
html_tables = soup.find_all('table', class_="wikitable plainrowheaders collapsible")

# Select the third table (index 2) containing launch records
first_launch_table = html_tables[2]

# Extract column names from the header row
header_row = first_launch_table.find('tr')
column_headers = header_row.find_all('th')
column_names = [extract_column_from_header(th) for th in column_headers if extract_column_from_header(th)]

# Print the column names
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


In [6]:
# Initialize dictionary with specific keys
launch_dict = {
    'Flight No.': [],
    'Launch site': [],
    'Payload': [],
    'Payload mass': [],
    'Orbit': [],
    'Customer': [],
    'Launch outcome': [],
    'Version Booster': [],
    'Booster landing': [],
    'Date': [],
    'Time': []
}

# Parse all tables and extract data
extracted_row = 0
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    for rows in table.find_all("tr"):
        # Check if the row has a flight number
        if rows.th and rows.th.string:
            flight_number = rows.th.string.strip()
            flag = flight_number.isdigit()
        else:
            flag = False
        
        # Extract data cells
        row = rows.find_all('td')
        if flag:
            extracted_row += 1
            
            # Flight Number
            launch_dict['Flight No.'].append(flight_number)
            
            # Date and Time
            datatimelist = date_time(row[0])
            launch_dict['Date'].append(datatimelist[0].strip(','))
            launch_dict['Time'].append(datatimelist[1])
            
            # Booster Version
            bv = booster_version(row[1])
            if not bv:
                bv = row[1].a.string if row[1].a else None
            launch_dict['Version Booster'].append(bv)
            
            # Launch Site
            launch_site = row[2].a.string if row[2].a else None
            launch_dict['Launch site'].append(launch_site)
            
            # Payload
            payload = row[3].a.string if row[3].a else None
            launch_dict['Payload'].append(payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            
            # Orbit
            orbit = row[5].a.string if row[5].a else None
            launch_dict['Orbit'].append(orbit)
            
            # Customer
            customer = row[6].a.string if row[6].a else None
            launch_dict['Customer'].append(customer)
            
            # Launch Outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Booster Landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

# Create DataFrame
df = pd.DataFrame(launch_dict)

# Display the first few rows
print(df.head())

# Save to CSV
df.to_csv('spacex_web_scraped.csv', index=False)

  Flight No. Launch site                               Payload Payload mass  \
0          1       CCAFS  Dragon Spacecraft Qualification Unit            0   
1          2       CCAFS                                Dragon            0   
2          3       CCAFS                                Dragon       525 kg   
3          4       CCAFS                          SpaceX CRS-1     4,700 kg   
4          5       CCAFS                          SpaceX CRS-2     4,877 kg   

  Orbit Customer Launch outcome   Version Booster Booster landing  \
0   LEO   SpaceX      Success\n  F9 v1.07B0003.18         Failure   
1   LEO     NASA        Success  F9 v1.07B0004.18         Failure   
2   LEO     NASA        Success  F9 v1.07B0005.18    No attempt\n   
3   LEO     NASA      Success\n  F9 v1.07B0006.18      No attempt   
4   LEO     NASA      Success\n  F9 v1.07B0007.18    No attempt\n   

              Date   Time  
0      4 June 2010  18:45  
1  8 December 2010  15:43  
2      22 May 2012  07:44 

## Summary

You have successfully:
1. **Requested** the Falcon 9 launch Wikipedia page using `requests`.
2. **Extracted** column names from the table header using `BeautifulSoup`.
3. **Parsed** launch data from all relevant tables and stored it in a DataFrame.
4. **Exported** the data to a CSV file named `spacex_web_scraped.csv`.

This data can now be used for further analysis, such as predicting Falcon 9 first-stage landing success.

## Authors

- [Aashu Baria](https://www.linkedin.com/in//)




